### TODO

- The drop-sampling of very frequent words has not yet been implemented.
- Skip-gram

In [1]:
import linguamind.linalg as la
import linguamind.nn as nn
import linguamind.nlp as nlp

from collections import Counter
import random

path ='/Users/amberedmundson/Laboratory/datasets/harryPotter/harryPotterAsciiCleaned.txt'
# path = '/Users/amberedmundson/Laboratory/clones/word2vec/text.txt'

v = nlp.Vocab()
t = nlp.Text(path,v)

v2 = nlp.Vocab(t.vocab)
v2.sort(10)

In [2]:
t.ChangeVocab(v2)
v2.InitUnigramTable()

vocab_size = v2.size
dim = 32

seed = la.Seed(1)

syn0 = nn.SparseLinearInput(vocab_size*6,dim)
syn0.weights.uniform(seed)
syn0.weights -= 0.5
syn0.weights /= dim

syn2 = nn.Linear(dim,dim)
syn2.weights.uniform(seed)
syn2.weights -= 0.5
syn2.weights /= dim

syn1neg = nn.SparseLinearOutput(dim,vocab_size)
syn1neg.weights.zero()

layers = list()
layers.append(syn0)
layers.append(syn1neg)
layers.append(nn.Sigmoid(vocab_size))

mlp = nn.Sequential(layers)
criterion = nn.MSECriterion()
optim = nn.StochasticGradient(mlp,criterion)

In [3]:
sampler = nn.Sampler(v2,5)

In [4]:
words = 0
for sent in t.sentences:
    for word in sent:
        words+=1

In [10]:
cbow = nn.CBOW(t.sentences,v2,sampler,3,0,True)

In [11]:
optim.train(cbow,0.01,1,1)

-1.0

In [12]:
%%timeit
optim.train(cbow,0.01,1,1)

1 loop, best of 3: 2.32 s per loop


In [9]:
%%timeit
optim.train(cbow,0.01,1,4)

1 loop, best of 3: 1.31 s per loop


In [ ]:
%%timeit
optim.train(cbow,0.01,3,4)

In [ ]:
words

In [8]:
import numpy as np

word2vec = {}
vocab = list()
for i in range(v2.size):
    l2 = np.sqrt(syn0.weights[i].dot(syn0.weights[i]))
    v = syn0.weights[i]
    v /= l2
    vocab.append(v2.getTermAtIndex(i).letters)
    word2vec[v2.getTermAtIndex(i).letters] = syn0.weights[i]
    
sim = Counter()
for word in vocab:
    sim[word] = word2vec['ron'].dot(word2vec[word])
sim.most_common()

[('ron', 0.9999999403953552),
 ('neville', 0.8440065383911133),
 ('krum', 0.787651002407074),
 ('lupin', 0.7704242467880249),
 ('james', 0.7573079466819763),
 ('dudley', 0.7472649812698364),
 ('hagrid', 0.7449439167976379),
 ('moody', 0.7444759011268616),
 ('seamus', 0.7432324886322021),
 ('bill', 0.7403506636619568),
 ('ginny', 0.7356343865394592),
 ('malfoy', 0.7235527634620667),
 ('fudge', 0.7222757339477539),
 ('riddle', 0.7074362635612488),
 ('karkaroff', 0.6989835500717163),
 ('tonks', 0.6961759328842163),
 ('fang', 0.6960095167160034),
 ('elf', 0.6920973658561707),
 ('snape', 0.6880075931549072),
 ('ernie', 0.6871569156646729),
 ('spider', 0.6865154504776001),
 ('goblin', 0.6825926303863525),
 ('bed', 0.6822382807731628),
 ('harry', 0.6811591386795044),
 ('griphook', 0.6786158084869385),
 ('bagman', 0.6777034401893616),
 ('breath', 0.6767666935920715),
 ('luna', 0.6765375137329102),
 ('lockhart', 0.674146831035614),
 ('fleur', 0.6739981174468994),
 ('crabbe', 0.6735944151878357)

# Forward Prop Modification for Windowing Layers

after you have confirmed that you are doing everything that word2vec does (exactly the same!!!)... such that you duplicate results, you should try this highly experimental thing

For CBOW, you should be able to tweak the output embedding based on how the inputs change and the previous gradient instead of regenerating it every time. If your window size is 5, this should give you close to a 5 * 2 = 10x speedup for the input window generation (which is where most of the parameters are going anyway)